# Python Processing Step for Multilabel FAIMS Data

### Packages

In [59]:
!conda env export --name tf21 > tensorflow21.yml

In [1]:
#Load all my packages
import timeit
import pandas as pd
import numpy as np
import numpy as np
from numpy import array
from numpy import argmax         #finds the index of the maximum value in a vector
import os
import sklearn
import sklearn.ensemble
from scipy import sparse
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from kerastuner.tuners import Hyperband
from tensorflow import keras
from skmultilearn.model_selection import IterativeStratification
from skmultilearn.model_selection import iterative_train_test_split, iterative_stratification
print(tf.__version__)

2.1.0


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


### Functions

### Processing

In [5]:
# read in data from R
df = pd.read_csv("P:/JGM_FAIMS_CVprediction/JMM_PreProcessed_Data/50percentMaxPlusThreshold.csv", low_memory=False) 
y = df.loc[ : ,  'X20':'X95'].values

In [6]:
df.head()

,SeqCharge,X20,X25,X30,X35,X40,X45,X50,X55,X60,...,X80,X85,X90,X95,maxcv_naomit,Charge,ModSequence,Length,Sequence,LabelSequence
0,2AACLCFR,0,0,0,0,0,0,1,1,1,...,0,0,0,0,50,2,AACLCFR,7,AACLCFR,1110000000
1,2SEQEDEVLLVSSSR,0,0,0,1,1,0,0,0,0,...,0,0,0,0,40,2,SEQEDEVLLVSSSR,14,SEQEDEVLLVSSSR,1100000000000
2,3SEQEDEVLLVSSSR,0,0,0,0,0,0,0,1,1,...,0,0,0,0,60,3,SEQEDEVLLVSSSR,14,SEQEDEVLLVSSSR,111100000
3,2YPDQWIVPGGGMEPEEEPGGAAVR,0,1,1,0,0,0,0,0,0,...,0,0,0,0,30,2,YPDQWIVPGGGMEPEEEPGGAAVR,24,YPDQWIVPGGGMEPEEEPGGAAVR,110000000000000
4,3YPDQWIVPGGGMEPEEEPGGAAVR,0,0,0,0,0,0,0,0,1,...,0,0,0,0,70,3,YPDQWIVPGGGMEPEEEPGGAAVR,24,YPDQWIVPGGGMEPEEEPGGAAVR,11100000


# Prepare the input to embedding layer - make array of index 

In [7]:
### combine all the letters into a long string, take the set to find the unique values, add 'END' (for use with one-hot), then get length
seq = df['SeqCharge']
vocab = set(''.join([str(i) for i in seq]))
vocab.add('END')
len_vocab = len(vocab)
print(len_vocab)
cv = df['maxcv_naomit']
set(cv)
## make index of the characters in vocab
char_index = dict((c, i) for i, c in enumerate(vocab))
maxlen = max([len(x) for x in df.SeqCharge])
print(char_index)
print(maxlen)

#take input upto max and truncate rest
# get index in char_index
#padd 'END' to shorter sequences

x = []
x_name = [str(i)[0:maxlen] for i in seq]
for i in x_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    x.append(tmp)
x = np.asarray(x)

27
{'2': 0, '3': 1, 'F': 2, 'a': 3, 'E': 4, 'T': 5, 'M': 6, '5': 7, 'm': 8, 'R': 9, 'END': 10, 'V': 11, 'A': 12, 'K': 13, 'I': 14, 'G': 15, 'W': 16, 'P': 17, 'Q': 18, 'D': 19, '4': 20, 'C': 21, 'N': 22, 'L': 23, 'S': 24, 'Y': 25, 'H': 26}
51


In [8]:
#Split the 50%+ threshold data into train and test keeping label distribution proportional
X_train, y_train, X_test, y_test = iterative_train_test_split(x, y, test_size=0.30)
X_train.shape

(85989, 51)

# Model

In [9]:
# F2 score THIS ONE IS WRONG 
import tensorflow.keras.backend as K

def f2(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 5*p*r / (4*p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [40]:
def fbeta2(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

# test the tuning in loop

In [10]:
import logging 
from kerastuner import Objective
logging.basicConfig(level=logging.DEBUG, filename="logfile_fix_trainset", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")

In [11]:
# complex model
METRICS = [
    keras.metrics.BinaryAccuracy(name='acc'),
    keras.metrics.Precision(name='prec'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    f2
    
]

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Embedding(input_dim=51, output_dim=hp.Int('embed',
                                        min_value=32,
                                        max_value=96,
                                        step=32)
                              ))
    model.add(layers.LSTM(hp.Int('lstm1',
                                        min_value=32,
                                        max_value=96,
                                        step=32), 
                          return_sequences=False, input_shape=(maxlen,len_vocab)))
    model.add(layers.Dropout(hp.Float('dropout1', min_value=0, max_value=0.5, sampling='linear', default=0.5)))
    model.add(layers.Dense(hp.Int('dense1',
                                        min_value=32,
                                        max_value=96,
                                        step=32), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout3', min_value=0, max_value=0.5, sampling='linear', default=0.5)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(16, activation='sigmoid'))
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 
                    min_value=1e-4, 
                    max_value=1e-2, 
                    sampling='LOG',
                    default=1e-3)),  
                  metrics=METRICS)
    return model


In [15]:

%%capture
### do only fold 0
# train loop, save predictions and models
start = timeit.default_timer()

threshacc = []
precision = []
recall = []
rocauc = []
f2_score = []
times = []

best_trial_dict = {}
model_dict = {}
results_dict={}


fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    if fold==0:
        logging.info('fold='+str(fold) )
        tuner = Hyperband(
            build_model,
            objective= Objective("val_auc", direction="max"),
            max_epochs=1000,
            hyperband_iterations=1,
            seed=23,
            directory='keras_tuner_kfold_6',
            project_name='hyperband'+str(fold)
            )
        logging.info(tuner.search_space_summary())
        tuner.search(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                     validation_data=(x[test], y[test]), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
                 verbose=0)
        best_trial_dict[fold]= tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
        model_dict[fold] = tuner.get_best_models(num_models=1)
        model = tuner.get_best_models(num_models=1)[0]

        #Train the multiforest using the training indices
        result=model.evaluate(x[test], y[test])

        results_dict[fold] = dict(zip(model.metrics_names, result))
        #record and print metrics
        f2_score.append(results_dict[fold]['f2'])
        threshacc. append(results_dict[fold]['acc'])
        precision.append(results_dict[fold]['prec'])
        rocauc.append(results_dict[fold]['auc'])
        recall.append(results_dict[fold]['recall'])
        logging.info('val_f2 =' + str(results_dict[fold]['f2']))
        logging.info('val_acc =' + str(results_dict[fold]['acc']))
        logging.info('val_prec =' + str(results_dict[fold]['prec']))
        logging.info('val_auc =' + str(results_dict[fold]['auc']))
        logging.info('val_recall =' + str(results_dict[fold]['recall']))
        fold+=1
        times.append(timeit.default_timer()-start)
stop = timeit.default_timer()

# get the runtime
run_time = stop - start
print('total run time = ' + str(run_time) )


INFO:tensorflow:Oracle triggered exit


In [16]:

%%capture
### do only fold 0
# train loop, save predictions and models


fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    if fold==1:
        logging.info('fold='+str(fold) )
        tuner = Hyperband(
            build_model,
            objective= Objective("val_auc", direction="max"),
            max_epochs=1000,
            hyperband_iterations=1,
            seed=23,
            directory='keras_tuner_kfold_6',
            project_name='hyperband'+str(fold)
            )
        logging.info(tuner.search_space_summary())
        tuner.search(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                     validation_data=(x[test], y[test]), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
                 verbose=0)
        
        best_trial_dict[fold]= tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
        logging.info(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        model = tuner.get_best_models(num_models=1)[0]
        model_dict[fold] = model
        #Train the multiforest using the training indices
        result=model.evaluate(x[test], y[test])

        results_dict[fold] = dict(zip(model.metrics_names, result))
        #record and print metrics
        f2_score.append(results_dict[fold]['f2'])
        threshacc. append(results_dict[fold]['acc'])
        precision.append(results_dict[fold]['prec'])
        rocauc.append(results_dict[fold]['auc'])
        recall.append(results_dict[fold]['recall'])
        logging.info('val_f2 =' + str(results_dict[fold]['f2']))
        logging.info('val_acc =' + str(results_dict[fold]['acc']))
        logging.info('val_prec =' + str(results_dict[fold]['prec']))
        logging.info('val_auc =' + str(results_dict[fold]['auc']))
        logging.info('val_recall =' + str(results_dict[fold]['recall']))
    fold+=1
    times.append(timeit.default_timer()-start)
stop = timeit.default_timer()

# get the runtime
run_time = stop - start
print('total run time = ' + str(run_time) )

INFO:tensorflow:Oracle triggered exit


In [35]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'embed': 64,
 'lstm1': 64,
 'dropout1': 0.15042412861806165,
 'dense1': 64,
 'dropout3': 0.031469519816518465,
 'learning_rate': 0.0024353059331955144,
 'tuner/epochs': 1000,
 'tuner/initial_epoch': 334,
 'tuner/bracket': 4,
 'tuner/round': 4,
 'tuner/trial_id': '23290a35acdfa07027cf99d0d203396a'}

In [36]:
best_trial_dict

{0: {'embed': 96,
  'lstm1': 96,
  'dropout1': 0.3906271577374155,
  'dense1': 64,
  'dropout3': 0.004887354825383772,
  'learning_rate': 0.0017580406699854202,
  'tuner/epochs': 334,
  'tuner/initial_epoch': 112,
  'tuner/bracket': 5,
  'tuner/round': 4,
  'tuner/trial_id': '34bb37723a74e88733c47827173bff56'},
 1: {'embed': 64,
  'lstm1': 64,
  'dropout1': 0.15042412861806165,
  'dense1': 64,
  'dropout3': 0.031469519816518465,
  'learning_rate': 0.0024353059331955144,
  'tuner/epochs': 1000,
  'tuner/initial_epoch': 334,
  'tuner/bracket': 4,
  'tuner/round': 4,
  'tuner/trial_id': '23290a35acdfa07027cf99d0d203396a'}}

In [20]:
model0 = model_dict[0][0]

In [29]:
model0.save('model0')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model0\assets


In [34]:
from tensorflow.python.saved_model import loader_impl
from tensorflow.python.keras.saving.saved_model import load as saved_model_load
load_path = 'model0'
loader_impl.parse_saved_model(load_path)
model = saved_model_load.load(load_path, custom_objects={"f2": f2})

TypeError: load() got an unexpected keyword argument 'custom_objects'

In [37]:

%%capture
### do only fold 0
# train loop, save predictions and models


fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    if fold==2:
        logging.info('fold='+str(fold) )
        tuner = Hyperband(
            build_model,
            objective= Objective("val_auc", direction="max"),
            max_epochs=1000,
            hyperband_iterations=1,
            seed=23,
            directory='keras_tuner_kfold_6',
            project_name='hyperband'+str(fold)
            )
        logging.info(tuner.search_space_summary())
        tuner.search(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                     validation_data=(x[test], y[test]), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
                 verbose=0)
        
        best_trial_dict[fold]= tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
        logging.info(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        model = tuner.get_best_models(num_models=1)[0]
        model_dict[fold] = model
        #Train the multiforest using the training indices
        result=model.evaluate(x[test], y[test])

        results_dict[fold] = dict(zip(model.metrics_names, result))
        #record and print metrics
        f2_score.append(results_dict[fold]['f2'])
        threshacc. append(results_dict[fold]['acc'])
        precision.append(results_dict[fold]['prec'])
        rocauc.append(results_dict[fold]['auc'])
        recall.append(results_dict[fold]['recall'])
        logging.info('val_f2 =' + str(results_dict[fold]['f2']))
        logging.info('val_acc =' + str(results_dict[fold]['acc']))
        logging.info('val_prec =' + str(results_dict[fold]['prec']))
        logging.info('val_auc =' + str(results_dict[fold]['auc']))
        logging.info('val_recall =' + str(results_dict[fold]['recall']))
    fold+=1
    times.append(timeit.default_timer()-start)
stop = timeit.default_timer()

# get the runtime
run_time = stop - start
print('total run time = ' + str(run_time) )

INFO:tensorflow:Oracle triggered exit


In [ ]:

%%capture
### do only fold 0
# train loop, save predictions and models


fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    if fold==3:
        logging.info('fold='+str(fold) )
        tuner = Hyperband(
            build_model,
            objective= Objective("val_auc", direction="max"),
            max_epochs=1000,
            hyperband_iterations=1,
            seed=23,
            directory='keras_tuner_kfold_6',
            project_name='hyperband'+str(fold)
            )
        logging.info(tuner.search_space_summary())
        tuner.search(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                     validation_data=(x[test], y[test]), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
                 verbose=0)
        
        best_trial_dict[fold]= tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
        logging.info(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        model = tuner.get_best_models(num_models=1)[0]
        model_dict[fold] = model
        #Train the multiforest using the training indices
        result=model.evaluate(x[test], y[test])

        results_dict[fold] = dict(zip(model.metrics_names, result))
        #record and print metrics
        f2_score.append(results_dict[fold]['f2'])
        threshacc. append(results_dict[fold]['acc'])
        precision.append(results_dict[fold]['prec'])
        rocauc.append(results_dict[fold]['auc'])
        recall.append(results_dict[fold]['recall'])
        logging.info('val_f2 =' + str(results_dict[fold]['f2']))
        logging.info('val_acc =' + str(results_dict[fold]['acc']))
        logging.info('val_prec =' + str(results_dict[fold]['prec']))
        logging.info('val_auc =' + str(results_dict[fold]['auc']))
        logging.info('val_recall =' + str(results_dict[fold]['recall']))
    fold+=1
    #times.append(timeit.default_timer()-start)
stop = timeit.default_timer()

# get the runtime
run_time = stop - start
print('total run time = ' + str(run_time) )

In [14]:

%%capture
### do only fold 0
# train loop, save predictions and models

start = timeit.default_timer()

threshacc = []
precision = []
recall = []
rocauc = []
f2_score = []
times = []

best_trial_dict = {}
model_dict = {}
results_dict={}


fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    if fold==4:
        logging.info('fold='+str(fold) )
        tuner = Hyperband(
            build_model,
            objective= Objective("val_auc", direction="max"),
            max_epochs=1000,
            hyperband_iterations=1,
            seed=23,
            directory='keras_tuner_kfold_6',
            project_name='hyperband'+str(fold)
            )
        logging.info(tuner.search_space_summary())
        tuner.search(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                     validation_data=(x[test], y[test]), 
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
                 verbose=0)
        
        best_trial_dict[fold]= tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
        logging.info(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        model = tuner.get_best_models(num_models=1)[0]
        model_dict[fold] = model
        #Train the multiforest using the training indices
        result=model.evaluate(x[test], y[test])
        results_dict[fold] = dict(zip(model.metrics_names, result))
        #record and print metrics
        f2_score.append(results_dict[fold]['f2'])
        threshacc. append(results_dict[fold]['acc'])
        precision.append(results_dict[fold]['prec'])
        rocauc.append(results_dict[fold]['auc'])
        recall.append(results_dict[fold]['recall'])
        logging.info('val_f2 =' + str(results_dict[fold]['f2']))
        logging.info('val_acc =' + str(results_dict[fold]['acc']))
        logging.info('val_prec =' + str(results_dict[fold]['prec']))
        logging.info('val_auc =' + str(results_dict[fold]['auc']))
        logging.info('val_recall =' + str(results_dict[fold]['recall']))
    fold+=1
    #times.append(timeit.default_timer()-start)
stop = timeit.default_timer()

# get the runtime
run_time = stop - start
print('total run time = ' + str(run_time) )

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_kfold_6\hyperband4\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner_kfold_6\hyperband4\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [15]:
 best_trial_dict

{4: {'embed': 96,
  'lstm1': 96,
  'dropout1': 0.2156097764514988,
  'dense1': 96,
  'dropout3': 0.02106432867132807,
  'learning_rate': 0.0026489221357360827,
  'tuner/epochs': 1000,
  'tuner/initial_epoch': 334,
  'tuner/bracket': 6,
  'tuner/round': 6,
  'tuner/trial_id': 'c3fb475f68fafd95147f903dc61c13e8'}}

In [16]:
### view the best parameter dictionary
logging.info('best trial hyperparameter summary')
for k in best_trial_dict.keys():
    logging.info(k)
    logging.info(best_trial_dict[k])
    
best_trial_dict

{4: {'embed': 96,
  'lstm1': 96,
  'dropout1': 0.2156097764514988,
  'dense1': 96,
  'dropout3': 0.02106432867132807,
  'learning_rate': 0.0026489221357360827,
  'tuner/epochs': 1000,
  'tuner/initial_epoch': 334,
  'tuner/bracket': 6,
  'tuner/round': 6,
  'tuner/trial_id': 'c3fb475f68fafd95147f903dc61c13e8'}}

In [27]:
tmpid = tuner.oracle.get_best_trials()[0].hyperparameters.values['tuner/trial_id']

'c3fb475f68fafd95147f903dc61c13e8'

In [28]:
tmp_trial = tuner.oracle.get_best_trials()[0]

# figure out how many epochs to do??

In [41]:
# best model params
METRICS = [
    keras.metrics.BinaryAccuracy(name='acc'),
    keras.metrics.Precision(name='prec'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    f2, 
    fbeta2
]

def build_model():
    model = tf.keras.Sequential()
    model.add(layers.Embedding(input_dim=51, output_dim=96))
    model.add(layers.LSTM(96, return_sequences=False, input_shape=(maxlen,len_vocab)))
    model.add(layers.Dropout(0.32255))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.02152))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(16, activation='sigmoid'))
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=0.001758),  
                  metrics=METRICS)
    return model


In [42]:
fold=0
k_fold = IterativeStratification(n_splits=5, order=3)#, random_state=23)
for train, test in k_fold.split(X_train, y_train):
    print(fold)
    model = build_model()
    hist = model.fit(x[train], y[train], epochs=1000, batch_size=1024, workers=4, 
                 validation_data=(x[test], y[test]), 
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max')], 
             verbose=2)

    #Train the multiforest using the training indices
    result=model.evaluate(x[test], y[test])
    results_dict = dict(zip(model.metrics_names, result))
    #record and print metrics

    fold+=1

0
Train on 68792 samples, validate on 17197 samples
Epoch 1/1000
68792/68792 - 5s - loss: 0.5567 - acc: 0.8297 - prec: 0.1405 - recall: 0.0658 - auc: 0.5264 - f2: 0.0372 - fbeta2: 0.0496 - val_loss: 0.3877 - val_acc: 0.8729 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7020 - val_f2: 0.0000e+00 - val_fbeta2: 6.0992e-11
Epoch 2/1000
68792/68792 - 2s - loss: 0.3509 - acc: 0.8726 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7236 - f2: 0.0000e+00 - fbeta2: 6.3877e-11 - val_loss: 0.3379 - val_acc: 0.8729 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7467 - val_f2: 0.0000e+00 - val_fbeta2: 6.0992e-11
Epoch 3/1000
68792/68792 - 2s - loss: 0.3353 - acc: 0.8725 - prec: 0.3907 - recall: 8.4148e-04 - auc: 0.7500 - f2: 5.8372e-04 - fbeta2: 0.0010 - val_loss: 0.3346 - val_acc: 0.8729 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7474 - val_f2: 0.0000e+00 - val_fbeta2: 6.0992e-11
Epoch 4/1000
68792/68792 - 2s - loss: 0.3340 - acc: 0.8726 - prec: 0.000

17197/17197 [==============================] - ETA: 5s - loss: 0.3240 - acc: 0.8730 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7737 - f2: 0.0000e+00 - fbeta2: 1.9231e-0 - ETA: 2s - loss: 0.3256 - acc: 0.8737 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7607 - f2: 0.0000e+00 - fbeta2: 1.9458e-0 - ETA: 2s - loss: 0.3277 - acc: 0.8736 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7567 - f2: 0.0000e+00 - fbeta2: 1.9411e-0 - ETA: 2s - loss: 0.3328 - acc: 0.8725 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7492 - f2: 0.0000e+00 - fbeta2: 1.9292e-0 - ETA: 2s - loss: 0.3331 - acc: 0.8722 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7503 - f2: 0.0000e+00 - fbeta2: 1.9224e-0 - ETA: 2s - loss: 0.3354 - acc: 0.8724 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7443 - f2: 0.0000e+00 - fbeta2: 1.9256e-0 - ETA: 2s - loss: 0.3362 - acc: 0.8721 - prec: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7437 - f2: 0.0000e+00 - fbeta2: 1.9204e-0 - ETA: 2s - loss: 0.3366 - acc: 0.8719 - prec

Epoch 8/1000
68791/68791 - 2s - loss: 0.2277 - acc: 0.8917 - prec: 0.6189 - recall: 0.3869 - auc: 0.9150 - f2: 0.2798 - fbeta2: 0.4183 - val_loss: 0.2236 - val_acc: 0.8924 - val_prec: 0.6433 - val_recall: 0.3568 - val_auc: 0.9195 - val_f2: 0.2726 - val_fbeta2: 0.3917
Epoch 9/1000
68791/68791 - 2s - loss: 0.2270 - acc: 0.8912 - prec: 0.6136 - recall: 0.3911 - auc: 0.9156 - f2: 0.2890 - fbeta2: 0.4218 - val_loss: 0.2420 - val_acc: 0.8836 - val_prec: 0.5800 - val_recall: 0.3270 - val_auc: 0.9044 - val_f2: 0.2496 - val_fbeta2: 0.3582
Epoch 10/1000
68791/68791 - 2s - loss: 0.2241 - acc: 0.8926 - prec: 0.6206 - recall: 0.3998 - auc: 0.9180 - f2: 0.2972 - fbeta2: 0.4307 - val_loss: 0.2253 - val_acc: 0.8940 - val_prec: 0.6394 - val_recall: 0.3937 - val_auc: 0.9176 - val_f2: 0.2785 - val_fbeta2: 0.4264
Epoch 11/1000
68791/68791 - 2s - loss: 0.2213 - acc: 0.8939 - prec: 0.6246 - recall: 0.4147 - auc: 0.9205 - f2: 0.3102 - fbeta2: 0.4451 - val_loss: 0.2182 - val_acc: 0.8952 - val_prec: 0.6496 - v

Epoch 39/1000
68791/68791 - 2s - loss: 0.1960 - acc: 0.9065 - prec: 0.6619 - recall: 0.5414 - auc: 0.9402 - f2: 0.4400 - fbeta2: 0.5617 - val_loss: 0.1984 - val_acc: 0.9058 - val_prec: 0.6711 - val_recall: 0.5174 - val_auc: 0.9389 - val_f2: 0.4060 - val_fbeta2: 0.5422
Epoch 40/1000
68791/68791 - 2s - loss: 0.1953 - acc: 0.9068 - prec: 0.6639 - recall: 0.5415 - auc: 0.9407 - f2: 0.4422 - fbeta2: 0.5620 - val_loss: 0.1988 - val_acc: 0.9049 - val_prec: 0.6434 - val_recall: 0.5760 - val_auc: 0.9388 - val_f2: 0.4840 - val_fbeta2: 0.5883
Epoch 41/1000
68791/68791 - 2s - loss: 0.1955 - acc: 0.9068 - prec: 0.6628 - recall: 0.5442 - auc: 0.9405 - f2: 0.4455 - fbeta2: 0.5648 - val_loss: 0.1993 - val_acc: 0.9051 - val_prec: 0.6672 - val_recall: 0.5155 - val_auc: 0.9382 - val_f2: 0.4272 - val_fbeta2: 0.5400
Epoch 42/1000
68791/68791 - 2s - loss: 0.1952 - acc: 0.9070 - prec: 0.6642 - recall: 0.5437 - auc: 0.9407 - f2: 0.4429 - fbeta2: 0.5636 - val_loss: 0.1988 - val_acc: 0.9052 - val_prec: 0.6577 -

17198/17198 [==============================] - ETA: 5s - loss: 0.2029 - acc: 0.9102 - prec: 0.6545 - recall: 0.5714 - auc: 0.9396 - f2: 0.4260 - fbeta2: 0.586 - ETA: 2s - loss: 0.2006 - acc: 0.9050 - prec: 0.6642 - recall: 0.5309 - auc: 0.9387 - f2: 0.3573 - fbeta2: 0.552 - ETA: 2s - loss: 0.2051 - acc: 0.9025 - prec: 0.6443 - recall: 0.5258 - auc: 0.9347 - f2: 0.3566 - fbeta2: 0.544 - ETA: 2s - loss: 0.2029 - acc: 0.9052 - prec: 0.6600 - recall: 0.5429 - auc: 0.9369 - f2: 0.3777 - fbeta2: 0.561 - ETA: 2s - loss: 0.2006 - acc: 0.9064 - prec: 0.6679 - recall: 0.5456 - auc: 0.9384 - f2: 0.3724 - fbeta2: 0.565 - ETA: 2s - loss: 0.1968 - acc: 0.9077 - prec: 0.6692 - recall: 0.5536 - auc: 0.9404 - f2: 0.3838 - fbeta2: 0.572 - ETA: 2s - loss: 0.1980 - acc: 0.9072 - prec: 0.6627 - recall: 0.5536 - auc: 0.9393 - f2: 0.3809 - fbeta2: 0.571 - ETA: 2s - loss: 0.1981 - acc: 0.9070 - prec: 0.6677 - recall: 0.5522 - auc: 0.9399 - f2: 0.3804 - fbeta2: 0.571 - ETA: 2s - loss: 0.1976 - acc: 0.9073 - pr

...

Epoch 42/1000
68791/68791 - 2s - loss: 0.1957 - acc: 0.9066 - prec: 0.6640 - recall: 0.5416 - auc: 0.9405 - f2: 0.4470 - fbeta2: 0.5628 - val_loss: 0.1972 - val_acc: 0.9065 - val_prec: 0.6672 - val_recall: 0.5250 - val_auc: 0.9391 - val_f2: 0.4310 - val_fbeta2: 0.5484
Epoch 43/1000
68791/68791 - 2s - loss: 0.1953 - acc: 0.9067 - prec: 0.6641 - recall: 0.5425 - auc: 0.9407 - f2: 0.4410 - fbeta2: 0.5626 - val_loss: 0.1980 - val_acc: 0.9058 - val_prec: 0.6514 - val_recall: 0.5551 - val_auc: 0.9386 - val_f2: 0.4400 - val_fbeta2: 0.5721
Epoch 44/1000
68791/68791 - 2s - loss: 0.1951 - acc: 0.9069 - prec: 0.6636 - recall: 0.5466 - auc: 0.9410 - f2: 0.4531 - fbeta2: 0.5670 - val_loss: 0.2001 - val_acc: 0.9048 - val_prec: 0.6475 - val_recall: 0.5488 - val_auc: 0.9370 - val_f2: 0.4504 - val_fbeta2: 0.5661
Epoch 45/1000
68791/68791 - 2s - loss: 0.1945 - acc: 0.9071 - prec: 0.6650 - recall: 0.5460 - auc: 0.9414 - f2: 0.4542 - fbeta2: 0.5662 - val_loss: 0.1990 - val_acc: 0.9052 - val_prec: 0.6447 -

17198/17198 [==============================] - ETA: 5s - loss: 0.1960 - acc: 0.9004 - prec: 0.6462 - recall: 0.6000 - auc: 0.9450 - f2: 0.3989 - fbeta2: 0.608 - ETA: 2s - loss: 0.2043 - acc: 0.9011 - prec: 0.6216 - recall: 0.5611 - auc: 0.9361 - f2: 0.4003 - fbeta2: 0.570 - ETA: 2s - loss: 0.2052 - acc: 0.9031 - prec: 0.6338 - recall: 0.5637 - auc: 0.9355 - f2: 0.3881 - fbeta2: 0.575 - ETA: 2s - loss: 0.2033 - acc: 0.9046 - prec: 0.6386 - recall: 0.5706 - auc: 0.9357 - f2: 0.3969 - fbeta2: 0.581 - ETA: 2s - loss: 0.2048 - acc: 0.9049 - prec: 0.6481 - recall: 0.5646 - auc: 0.9356 - f2: 0.4001 - fbeta2: 0.577 - ETA: 2s - loss: 0.2043 - acc: 0.9061 - prec: 0.6549 - recall: 0.5724 - auc: 0.9363 - f2: 0.4072 - fbeta2: 0.585 - ETA: 2s - loss: 0.2030 - acc: 0.9064 - prec: 0.6579 - recall: 0.5735 - auc: 0.9370 - f2: 0.4128 - fbeta2: 0.587 - ETA: 2s - loss: 0.2021 - acc: 0.9068 - prec: 0.6566 - recall: 0.5769 - auc: 0.9373 - f2: 0.4167 - fbeta2: 0.590 - ETA: 2s - loss: 0.2008 - acc: 0.9073 - pr

Epoch 11/1000
68791/68791 - 2s - loss: 0.2251 - acc: 0.8916 - prec: 0.6160 - recall: 0.3954 - auc: 0.9174 - f2: 0.2977 - fbeta2: 0.4254 - val_loss: 0.2305 - val_acc: 0.8885 - val_prec: 0.5898 - val_recall: 0.4102 - val_auc: 0.9124 - val_f2: 0.3333 - val_fbeta2: 0.4366
Epoch 12/1000
68791/68791 - 2s - loss: 0.2209 - acc: 0.8938 - prec: 0.6280 - recall: 0.4069 - auc: 0.9210 - f2: 0.3086 - fbeta2: 0.4381 - val_loss: 0.2221 - val_acc: 0.8946 - val_prec: 0.6481 - val_recall: 0.3782 - val_auc: 0.9198 - val_f2: 0.2925 - val_fbeta2: 0.4123
Epoch 13/1000
68791/68791 - 2s - loss: 0.2203 - acc: 0.8939 - prec: 0.6255 - recall: 0.4154 - auc: 0.9213 - f2: 0.3181 - fbeta2: 0.4456 - val_loss: 0.2175 - val_acc: 0.8962 - val_prec: 0.6332 - val_recall: 0.4405 - val_auc: 0.9238 - val_f2: 0.3356 - val_fbeta2: 0.4688
Epoch 14/1000
68791/68791 - 2s - loss: 0.2188 - acc: 0.8944 - prec: 0.6258 - recall: 0.4234 - auc: 0.9227 - f2: 0.3229 - fbeta2: 0.4528 - val_loss: 0.2230 - val_acc: 0.8929 - val_prec: 0.6316 -

Epoch 42/1000
68791/68791 - 2s - loss: 0.1972 - acc: 0.9057 - prec: 0.6601 - recall: 0.5347 - auc: 0.9395 - f2: 0.4375 - fbeta2: 0.5557 - val_loss: 0.2102 - val_acc: 0.8987 - val_prec: 0.6301 - val_recall: 0.4973 - val_auc: 0.9297 - val_f2: 0.3761 - val_fbeta2: 0.5188
Epoch 43/1000
68791/68791 - 2s - loss: 0.1979 - acc: 0.9053 - prec: 0.6579 - recall: 0.5337 - auc: 0.9390 - f2: 0.4426 - fbeta2: 0.5547 - val_loss: 0.2039 - val_acc: 0.9028 - val_prec: 0.6397 - val_recall: 0.5430 - val_auc: 0.9347 - val_f2: 0.4720 - val_fbeta2: 0.5597
Epoch 44/1000
68791/68791 - 2s - loss: 0.1965 - acc: 0.9061 - prec: 0.6610 - recall: 0.5394 - auc: 0.9399 - f2: 0.4346 - fbeta2: 0.5596 - val_loss: 0.2012 - val_acc: 0.9045 - val_prec: 0.6485 - val_recall: 0.5477 - val_auc: 0.9366 - val_f2: 0.4074 - val_fbeta2: 0.5650
Epoch 45/1000
68791/68791 - 2s - loss: 0.1955 - acc: 0.9065 - prec: 0.6615 - recall: 0.5446 - auc: 0.9406 - f2: 0.4483 - fbeta2: 0.5652 - val_loss: 0.2032 - val_acc: 0.9035 - val_prec: 0.6519 -

Epoch 73/1000
68791/68791 - 2s - loss: 0.1819 - acc: 0.9147 - prec: 0.6848 - recall: 0.6110 - auc: 0.9496 - f2: 0.5224 - fbeta2: 0.6238 - val_loss: 0.1990 - val_acc: 0.9066 - val_prec: 0.6466 - val_recall: 0.5882 - val_auc: 0.9387 - val_f2: 0.4679 - val_fbeta2: 0.5988
Epoch 74/1000
68791/68791 - 2s - loss: 0.1811 - acc: 0.9148 - prec: 0.6852 - recall: 0.6122 - auc: 0.9501 - f2: 0.5207 - fbeta2: 0.6254 - val_loss: 0.2015 - val_acc: 0.9056 - val_prec: 0.6407 - val_recall: 0.5898 - val_auc: 0.9374 - val_f2: 0.4939 - val_fbeta2: 0.5992
Epoch 75/1000
68791/68791 - 2s - loss: 0.1809 - acc: 0.9149 - prec: 0.6844 - recall: 0.6148 - auc: 0.9502 - f2: 0.5274 - fbeta2: 0.6277 - val_loss: 0.1985 - val_acc: 0.9061 - val_prec: 0.6493 - val_recall: 0.5723 - val_auc: 0.9387 - val_f2: 0.4612 - val_fbeta2: 0.5861
17198/17198 [==============================] - ETA: 5s - loss: 0.2613 - acc: 0.8828 - prec: 0.5263 - recall: 0.4762 - auc: 0.8899 - f2: 0.3582 - fbeta2: 0.485 - ETA: 2s - loss: 0.1952 - acc: 0.

# train final model

In [43]:
model = build_model()
hist = model.fit(X_train, y_train, epochs=52, batch_size=1024, workers=4, 
             validation_data=(X_test, y_test), verbose=2)

#Train the multiforest using the training indices
result=model.evaluate(X_test, y_test)
results_dict = dict(zip(model.metrics_names, result))
results_dict


Train on 85989 samples, validate on 36858 samples
Epoch 1/52
85989/85989 - 5s - loss: 0.5211 - acc: 0.8388 - prec: 0.1396 - recall: 0.0514 - auc: 0.5277 - f2: 0.0296 - fbeta2: 0.0391 - val_loss: 0.3611 - val_acc: 0.8727 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7135 - val_f2: 0.0000e+00 - val_fbeta2: 5.9935e-11
Epoch 2/52
85989/85989 - 3s - loss: 0.3431 - acc: 0.8726 - prec: 1.0000 - recall: 5.7071e-06 - auc: 0.7357 - f2: 3.1184e-06 - fbeta2: 7.0552e-06 - val_loss: 0.3365 - val_acc: 0.8727 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7474 - val_f2: 0.0000e+00 - val_fbeta2: 5.9935e-11
Epoch 3/52
85989/85989 - 3s - loss: 0.3355 - acc: 0.8726 - prec: 0.2714 - recall: 1.0844e-04 - auc: 0.7473 - f2: 7.9933e-05 - fbeta2: 1.3533e-04 - val_loss: 0.3348 - val_acc: 0.8727 - val_prec: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7474 - val_f2: 0.0000e+00 - val_fbeta2: 5.9935e-11
Epoch 4/52
85989/85989 - 3s - loss: 0.3346 - acc: 0.8726 - prec: 0.0000e+00 - re

Epoch 31/52
85989/85989 - 3s - loss: 0.1984 - acc: 0.9056 - prec: 0.6618 - recall: 0.5296 - auc: 0.9385 - f2: 0.4334 - fbeta2: 0.5515 - val_loss: 0.2014 - val_acc: 0.9035 - val_prec: 0.6439 - val_recall: 0.5419 - val_auc: 0.9364 - val_f2: 0.4731 - val_fbeta2: 0.5596
Epoch 32/52
85989/85989 - 3s - loss: 0.1985 - acc: 0.9057 - prec: 0.6618 - recall: 0.5315 - auc: 0.9385 - f2: 0.4375 - fbeta2: 0.5531 - val_loss: 0.1989 - val_acc: 0.9054 - val_prec: 0.6525 - val_recall: 0.5509 - val_auc: 0.9381 - val_f2: 0.4242 - val_fbeta2: 0.5685
Epoch 33/52
85989/85989 - 3s - loss: 0.1972 - acc: 0.9063 - prec: 0.6639 - recall: 0.5350 - auc: 0.9393 - f2: 0.4372 - fbeta2: 0.5565 - val_loss: 0.1981 - val_acc: 0.9057 - val_prec: 0.6623 - val_recall: 0.5290 - val_auc: 0.9387 - val_f2: 0.4079 - val_fbeta2: 0.5510
Epoch 34/52
85989/85989 - 3s - loss: 0.1963 - acc: 0.9070 - prec: 0.6656 - recall: 0.5425 - auc: 0.9401 - f2: 0.4490 - fbeta2: 0.5631 - val_loss: 0.1973 - val_acc: 0.9063 - val_prec: 0.6618 - val_rec

26784/36858 [====================>.........] - ETA: 11s - loss: 0.2468 - acc: 0.8770 - prec: 0.5789 - recall: 0.4583 - auc: 0.9150 - f2: 0.4087 - fbeta2: 0.47 - ETA: 5s - loss: 0.2034 - acc: 0.9067 - prec: 0.6596 - recall: 0.5514 - auc: 0.9351 - f2: 0.4045 - fbeta2: 0.5719 - ETA: 5s - loss: 0.1998 - acc: 0.9103 - prec: 0.6754 - recall: 0.5575 - auc: 0.9372 - f2: 0.3997 - fbeta2: 0.578 - ETA: 5s - loss: 0.1976 - acc: 0.9093 - prec: 0.6736 - recall: 0.5540 - auc: 0.9392 - f2: 0.3983 - fbeta2: 0.574 - ETA: 5s - loss: 0.1980 - acc: 0.9089 - prec: 0.6697 - recall: 0.5569 - auc: 0.9387 - f2: 0.3980 - fbeta2: 0.575 - ETA: 4s - loss: 0.1995 - acc: 0.9066 - prec: 0.6570 - recall: 0.5507 - auc: 0.9375 - f2: 0.3933 - fbeta2: 0.568 - ETA: 4s - loss: 0.1977 - acc: 0.9071 - prec: 0.6604 - recall: 0.5536 - auc: 0.9390 - f2: 0.3941 - fbeta2: 0.571 - ETA: 4s - loss: 0.1983 - acc: 0.9065 - prec: 0.6576 - recall: 0.5512 - auc: 0.9384 - f2: 0.3932 - fbeta2: 0.568 - ETA: 4s - loss: 0.1989 - acc: 0.9064 - p

36858/36858 [==============================] - ETA: 1s - loss: 0.1927 - acc: 0.9084 - prec: 0.6645 - recall: 0.5621 - auc: 0.9425 - f2: 0.4062 - fbeta2: 0.579 - ETA: 1s - loss: 0.1926 - acc: 0.9084 - prec: 0.6648 - recall: 0.5622 - auc: 0.9425 - f2: 0.4066 - fbeta2: 0.579 - ETA: 1s - loss: 0.1927 - acc: 0.9084 - prec: 0.6647 - recall: 0.5622 - auc: 0.9425 - f2: 0.4066 - fbeta2: 0.579 - ETA: 1s - loss: 0.1928 - acc: 0.9083 - prec: 0.6643 - recall: 0.5620 - auc: 0.9424 - f2: 0.4064 - fbeta2: 0.579 - ETA: 1s - loss: 0.1929 - acc: 0.9083 - prec: 0.6642 - recall: 0.5620 - auc: 0.9423 - f2: 0.4068 - fbeta2: 0.579 - ETA: 1s - loss: 0.1929 - acc: 0.9083 - prec: 0.6646 - recall: 0.5624 - auc: 0.9423 - f2: 0.4073 - fbeta2: 0.579 - ETA: 1s - loss: 0.1932 - acc: 0.9082 - prec: 0.6643 - recall: 0.5620 - auc: 0.9422 - f2: 0.4070 - fbeta2: 0.579 - ETA: 0s - loss: 0.1932 - acc: 0.9082 - prec: 0.6639 - recall: 0.5621 - auc: 0.9422 - f2: 0.4071 - fbeta2: 0.579 - ETA: 0s - loss: 0.1932 - acc: 0.9082 - pr

{'loss': 0.19521527745990358,
 'acc': 0.9072674,
 'prec': 0.66095227,
 'recall': 0.5580575,
 'auc': 0.9408676,
 'f2': 0.40420324,
 'fbeta2': 0.57525325}

In [44]:
model.save('final_model.h5')


In [56]:
mload = keras.models.load_model('final_model_human_train.h5', custom_objects={'f2':f2, 'fbeta2':fbeta2})

In [57]:
results = mload.evaluate(X_test, y_test)
results_dict = dict(zip(mload.metrics_names, result))
results_dict

13568/36858 [==========>...................] - ETA: 9:43 - loss: 0.2468 - acc: 0.8770 - prec: 0.5789 - recall: 0.4583 - auc: 0.9150 - f2: 0.4087 - fbeta2: 0.478 - ETA: 1:47 - loss: 0.2126 - acc: 0.8971 - prec: 0.6136 - recall: 0.5293 - auc: 0.9287 - f2: 0.4006 - fbeta2: 0.546 - ETA: 58s - loss: 0.2039 - acc: 0.9054 - prec: 0.6540 - recall: 0.5479 - auc: 0.9349 - f2: 0.3940 - fbeta2: 0.568 - ETA: 42s - loss: 0.2067 - acc: 0.9064 - prec: 0.6552 - recall: 0.5405 - auc: 0.9319 - f2: 0.3953 - fbeta2: 0.56 - ETA: 34s - loss: 0.2014 - acc: 0.9095 - prec: 0.6737 - recall: 0.5524 - auc: 0.9361 - f2: 0.3950 - fbeta2: 0.57 - ETA: 29s - loss: 0.1992 - acc: 0.9095 - prec: 0.6750 - recall: 0.5538 - auc: 0.9378 - f2: 0.3974 - fbeta2: 0.57 - ETA: 26s - loss: 0.1980 - acc: 0.9091 - prec: 0.6730 - recall: 0.5550 - auc: 0.9390 - f2: 0.3989 - fbeta2: 0.57 - ETA: 23s - loss: 0.1980 - acc: 0.9092 - prec: 0.6716 - recall: 0.5542 - auc: 0.9385 - f2: 0.3978 - fbeta2: 0.57 - ETA: 21s - loss: 0.1980 - acc: 0.908

27232/36858 [=====================>........] - ETA: 7s - loss: 0.1942 - acc: 0.9077 - prec: 0.6589 - recall: 0.5604 - auc: 0.9411 - f2: 0.4063 - fbeta2: 0.576 - ETA: 7s - loss: 0.1943 - acc: 0.9076 - prec: 0.6585 - recall: 0.5603 - auc: 0.9411 - f2: 0.4062 - fbeta2: 0.576 - ETA: 7s - loss: 0.1942 - acc: 0.9076 - prec: 0.6583 - recall: 0.5604 - auc: 0.9411 - f2: 0.4061 - fbeta2: 0.576 - ETA: 7s - loss: 0.1942 - acc: 0.9076 - prec: 0.6581 - recall: 0.5603 - auc: 0.9411 - f2: 0.4066 - fbeta2: 0.576 - ETA: 7s - loss: 0.1942 - acc: 0.9076 - prec: 0.6583 - recall: 0.5599 - auc: 0.9412 - f2: 0.4069 - fbeta2: 0.576 - ETA: 7s - loss: 0.1942 - acc: 0.9075 - prec: 0.6581 - recall: 0.5596 - auc: 0.9412 - f2: 0.4069 - fbeta2: 0.575 - ETA: 7s - loss: 0.1940 - acc: 0.9076 - prec: 0.6587 - recall: 0.5602 - auc: 0.9413 - f2: 0.4072 - fbeta2: 0.576 - ETA: 7s - loss: 0.1940 - acc: 0.9075 - prec: 0.6587 - recall: 0.5598 - auc: 0.9413 - f2: 0.4073 - fbeta2: 0.576 - ETA: 6s - loss: 0.1939 - acc: 0.9075 - pr

36858/36858 [==============================] - ETA: 2s - loss: 0.1927 - acc: 0.9084 - prec: 0.6648 - recall: 0.5621 - auc: 0.9425 - f2: 0.4063 - fbeta2: 0.579 - ETA: 2s - loss: 0.1926 - acc: 0.9084 - prec: 0.6649 - recall: 0.5622 - auc: 0.9425 - f2: 0.4066 - fbeta2: 0.579 - ETA: 2s - loss: 0.1926 - acc: 0.9084 - prec: 0.6648 - recall: 0.5622 - auc: 0.9425 - f2: 0.4068 - fbeta2: 0.579 - ETA: 2s - loss: 0.1927 - acc: 0.9084 - prec: 0.6647 - recall: 0.5623 - auc: 0.9425 - f2: 0.4067 - fbeta2: 0.579 - ETA: 2s - loss: 0.1927 - acc: 0.9084 - prec: 0.6646 - recall: 0.5622 - auc: 0.9425 - f2: 0.4068 - fbeta2: 0.579 - ETA: 2s - loss: 0.1928 - acc: 0.9083 - prec: 0.6644 - recall: 0.5622 - auc: 0.9424 - f2: 0.4064 - fbeta2: 0.579 - ETA: 2s - loss: 0.1928 - acc: 0.9083 - prec: 0.6643 - recall: 0.5620 - auc: 0.9424 - f2: 0.4066 - fbeta2: 0.579 - ETA: 2s - loss: 0.1928 - acc: 0.9083 - prec: 0.6642 - recall: 0.5620 - auc: 0.9424 - f2: 0.4069 - fbeta2: 0.579 - ETA: 2s - loss: 0.1929 - acc: 0.9083 - pr

{'loss': 0.19521527745990358,
 'acc': 0.9072674,
 'prec': 0.66095227,
 'recall': 0.5580575,
 'auc': 0.9408676,
 'f2': 0.40420324,
 'fbeta2': 0.57525325}

# train the model on all human data

In [61]:
model = build_model()
hist = model.fit(x, y, epochs=52, batch_size=1024, workers=4, 
             validation_data=(), verbose=2)



Train on 122847 samples
Epoch 1/52
122847/122847 - 5s - loss: 0.4669 - acc: 0.8493 - prec: 0.1372 - recall: 0.0347 - auc: 0.5610 - f2: 0.0204 - fbeta2: 0.0262
Epoch 2/52
122847/122847 - 3s - loss: 0.2815 - acc: 0.8770 - prec: 0.5825 - recall: 0.1203 - auc: 0.8587 - f2: 0.1004 - fbeta2: 0.1406
Epoch 3/52
122847/122847 - 3s - loss: 0.2481 - acc: 0.8850 - prec: 0.6040 - recall: 0.2816 - auc: 0.8960 - f2: 0.2077 - fbeta2: 0.3146
Epoch 4/52
122847/122847 - 3s - loss: 0.2322 - acc: 0.8897 - prec: 0.6133 - recall: 0.3626 - auc: 0.9113 - f2: 0.2631 - fbeta2: 0.3946
Epoch 5/52
122847/122847 - 3s - loss: 0.2267 - acc: 0.8917 - prec: 0.6177 - recall: 0.3923 - auc: 0.9160 - f2: 0.2899 - fbeta2: 0.4229
Epoch 6/52
122847/122847 - 3s - loss: 0.2229 - acc: 0.8929 - prec: 0.6204 - recall: 0.4090 - auc: 0.9192 - f2: 0.3082 - fbeta2: 0.4385
Epoch 7/52
122847/122847 - 3s - loss: 0.2198 - acc: 0.8942 - prec: 0.6251 - recall: 0.4224 - auc: 0.9218 - f2: 0.3232 - fbeta2: 0.4514
Epoch 8/52
122847/122847 - 3s -

In [62]:
model.save('final_all_human_model.h5')